In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [5]:
df = pd.read_csv('./JudgeTest.csv',sep=',', encoding = 'utf-8')

In [18]:
type(y)

pandas.core.series.Series

In [17]:
y.shape

(11434,)

In [6]:
df = df.astype(int)
df

,record,location,compromise,content,education,mindset,financial,support,attitude,confess,sequel,crime,label
0,1,0,0,2,0,0,0,0,0,0,0,0,9
1,0,1,0,0,0,0,0,1,2,0,0,0,9
2,0,0,0,0,0,0,0,0,0,0,0,0,7
3,0,0,0,1,1,0,0,0,0,0,1,0,7
4,0,0,0,0,1,0,0,0,1,0,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0,0,0,2,1,0,0,1,0,0,1,0,5
195,0,0,0,2,0,0,0,0,0,1,0,1,5
196,0,0,0,0,0,0,1,1,0,0,1,1,5
197,0,1,0,0,0,0,1,0,0,0,0,1,5


In [6]:
pd.set_option('display.max_rows', 800)

In [12]:
df['label_i'].value_counts()

0     1547
1     1348
2     1267
3     1187
4      969
5      658
6      598
7      597
8      533
9      443
10     408
11     377
12     220
13     216
14     165
15     125
16     116
17     115
18     105
19     103
20      73
21      66
22      49
23      36
24      32
25      28
26      21
27      17
28      15
29       8
31       6
32       6
30       6
33       6
Name: label_i, dtype: int64

In [8]:
# 先用GridSearchCV找最適合的參數
# 準備要跑GridSearchCV的資料
X = df.drop(['label'],axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# 有做正規化的準確性比較高
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
%%time
# 設定要調整的參數
parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),'C':(1,10,100,1000)}
# parameters = {'C':(1,10,100,1000)}
# 建立空模
model = SVC()
# 將空模跟參數餵進GridSearchCV
clf = GridSearchCV(model, parameters)
# 餵data
clf.fit(X_train, y_train)
# 印出最佳參數
print(clf.best_estimator_)
print(clf.best_params_)
print(clf.best_score_)
# 看詳細result
#pd.DataFrame(clf.cv_results_)

SVC(C=1000)
{'C': 1000, 'kernel': 'rbf'}
0.5663306451612903
Wall time: 931 ms


In [11]:
%%time
# No Ensemble

# we can change kernel to rbf, poly, linear

model = SVC(kernel='rbf', C=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('con_matrix: {}'.format(con_matrix))

number of correct sample: 18
accuracy: 0.45
con_matrix: [[2 0 0 0 4 0 0 0 0 0]
 [1 5 0 0 0 0 0 0 0 0]
 [0 1 2 0 2 0 0 0 0 0]
 [0 1 0 2 0 0 1 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 0 1 3 0 0]
 [1 0 0 0 3 1 0 0 0 0]
 [0 0 0 0 2 0 0 1 0 1]]
Wall time: 4.99 ms


In [32]:
%%time
# Bagging

# we can change kernel to rbf, poly, linear

model = SVC(kernel='rbf', C=1)
ensemble = BaggingClassifier(base_estimator=model,
                            n_estimators=10)

ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('con_matrix: {}'.format(con_matrix))

number of correct sample: 405
accuracy: 0.17708788806296458
con_matrix: [[ 91  80  77  74   2   4   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 39 111  15 107   0   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 57  58  87  40   0   1   2   6   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 30  87   9 109   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 49  60  24  55   1   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 16  52  10  48   2   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 34  11  75   5   1   0   1   2   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [ 29   6  75   7   0   0   2   4   0   0   0   0   0   1   0   0   0   0
    0  

In [27]:
# Adaboost
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# we can change kernel to rbf, poly, linear

# model = SVC(kernel='poly', C=89)

# ensemble = AdaBoostClassifier(model,
#                               algorithm="SAMME",
#                               n_estimators=1000)

ensemble = AdaBoostClassifier(algorithm="SAMME",
                              n_estimators=20)

ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('con_matrix: {}'.format(con_matrix))

number of correct sample: 390
accuracy: 0.1705290773939659
con_matrix: [[  0   0 109 182   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  20 235   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 163 131   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  13 227   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  38 166   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  14 131   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  73  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  93  28   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   

In [31]:
from sklearn.ensemble import RandomForestClassifier

X = df.drop(['label'],axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# we can change kernel to rbf, poly, linear

model = RandomForestClassifier(n_estimators=170, max_depth=1)

ensemble = AdaBoostClassifier(model,
                              algorithm="SAMME",
                              n_estimators=100)

ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('con_matrix: {}'.format(con_matrix))

number of correct sample: 13
accuracy: 0.325
con_matrix: [[2 0 0 0 2 0 0 0 0 1]
 [1 2 0 1 0 1 0 0 0 1]
 [1 0 1 0 1 0 0 0 0 0]
 [1 4 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 1 0 1]
 [1 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 2 0 0 1 1 0]
 [0 0 0 0 1 0 0 4 1 0]
 [0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 1]]


In [12]:
# 這邊是假設放新的資料進去的樣態
z = np.array([1,1,1,2,1,1,1,1,0,0,1,0])
# 一定要reshape才可以丟模型
z = z.reshape(1, -1)
# 模型建立的時候featrue有正規化，所以後面送的數據也需要正規化
z = scaler.transform(z)
y_test = model.predict(z)
print(y_test)

[1]


C:\Users\Student\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
